<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/cookbooks/GraphRAG_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GraphRAG Implementation with LlamaIndex

[GraphRAG (Graphs + Retrieval Augmented Generation)](https://www.microsoft.com/en-us/research/project/graphrag/) combines the 

## GraphRAG Aproach

The GraphRAG involves two steps:

1. Graph Generation - Creates Graph, builds communities and its summaries over the given document.
2. Answer to the Query - Use summaries of the communities created from step-1 to answer the query.

**Graph Generation:**

1. **Source Documents to Text Chunks:** Source documents are divided into smaller text chunks for easier processing.

2. **Text Chunks to Element Instances:** Each text chunk is analyzed to identify and extract entities and relationships, resulting in a list of tuples that represent these elements.

3. **Element Instances to Element Summaries:** The extracted entities and relationships are summarized into descriptive text blocks for each element using the LLM.

4. **Element Summaries to Graph Communities:** These entities, relationships and summaries form a graph, which is subsequently partitioned into communities using algorithms using Heirarchical Leiden to establish a hierarchical structure.

5. **Graph Communities to Community Summaries:** The LLM generates summaries for each community, providing insights into the dataset’s overall topical structure and semantics.

**Answering the Query:**

**Community Summaries to Global Answers:** The summaries of the communities are utilized to respond to user queries. This involves generating intermediate answers, which are then consolidated into a comprehensive global answer.


## GraphRAG Pipeline Components

Here are the different components we implemented to build all of the processes mentioned above.

1. **Source Documents to Text Chunks:** Implemented using `SentenceSplitter` with a chunk size of 1024 and chunk overlap of 20 tokens.

2. **Text Chunks to Element Instances AND Element Instances to Element Summaries:** Implemented using `GraphRAGExtractor`.

3. **Element Summaries to Graph Communities AND Graph Communities to Community Summaries:** Implemented using `GraphRAGStore`.

4. **Community Summaries to Global Answers:** Implemented using `GraphQueryEngine`.


Let's check into each of these components and build GraphRAG pipeline.


## Installation

`graspologic` is used to use hierarchical_leiden for building communities.

In [26]:
!pip install llama-index graspologic numpy==1.24.4 scipy==1.12.0

## Load Data

We will use a sample news article dataset retrieved from Diffbot, which Tomaz has conveniently made available on GitHub for easy access.

The dataset contains 2,500 samples; for ease of experimentation, we will use 50 of these samples, which include the `title` and `text` of news articles.

Prepare documents as required by LlamaIndex

In [27]:
from dotenv import load_dotenv
import os
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings

import nest_asyncio
nest_asyncio.apply()

# Load the .env file

load_dotenv('../.env')

# Access the OpenAI key
openai_key = os.getenv("OPENAI_API_KEY")

llm = OpenAI(model="gpt-4o-mini", api_key=openai_key)
embed_model = OpenAIEmbedding(model="text-embedding-3-small")

Settings.llm = llm
Settings.embed_model = embed_model


In [44]:
import asyncio
import nest_asyncio

nest_asyncio.apply()
%load_ext autoreload
%autoreload 2

from typing import Any, List, Callable, Optional, Union, Dict
from IPython.display import Markdown, display

from llama_index.core.async_utils import run_jobs
from llama_index.core.indices.property_graph.utils import (
    default_parse_triplets_fn,
)
from llama_index.core.graph_stores.types import (
    EntityNode,
    KG_NODES_KEY,
    KG_RELATIONS_KEY,
    Relation,
)
from llama_index.core.llms.llm import LLM
from llama_index.core.prompts import PromptTemplate
from llama_index.core.prompts.default_prompts import (
    DEFAULT_KG_TRIPLET_EXTRACT_PROMPT,
)
from llama_index.core.schema import TransformComponent, BaseNode
from llama_index.core.bridge.pydantic import BaseModel, Field


class GraphRAGExtractor(TransformComponent):
    """Extract triples from a graph.

    Uses an LLM and a simple prompt + output parsing to extract paths (i.e. triples) and entity, relation descriptions from text.

    Args:
        llm (LLM):
            The language model to use.
        extract_prompt (Union[str, PromptTemplate]):
            The prompt to use for extracting triples.
        parse_fn (callable):
            A function to parse the output of the language model.
        num_workers (int):
            The number of workers to use for parallel processing.
        max_paths_per_chunk (int):
            The maximum number of paths to extract per chunk.
    """

    llm: LLM
    extract_prompt: PromptTemplate
    parse_fn: Callable
    num_workers: int
    max_paths_per_chunk: int

    def __init__(
        self,
        llm: Optional[LLM] = None,
        extract_prompt: Optional[Union[str, PromptTemplate]] = None,
        parse_fn: Callable = default_parse_triplets_fn,
        max_paths_per_chunk: int = 10,
        num_workers: int = 4,
    ) -> None:
        """Init params."""
        from llama_index.core import Settings

        if isinstance(extract_prompt, str):
            extract_prompt = PromptTemplate(extract_prompt)

        super().__init__(
            llm=llm or Settings.llm,
            extract_prompt=extract_prompt or DEFAULT_KG_TRIPLET_EXTRACT_PROMPT,
            parse_fn=parse_fn,
            num_workers=num_workers,
            max_paths_per_chunk=max_paths_per_chunk,
        )

    @classmethod
    def class_name(cls) -> str:
        return "GraphExtractor"

    def __call__(
        self, nodes: List[BaseNode], show_progress: bool = False, **kwargs: Any
    ) -> List[BaseNode]:
        """Extract triples from nodes."""
        return asyncio.run(
            self.acall(nodes, show_progress=show_progress, **kwargs)
        )

    async def _aextract(self, node: BaseNode) -> BaseNode:
        """Extract triples from a node."""
        assert hasattr(node, "text")

        text = node.get_content(metadata_mode="llm")
        try:
            llm_response = await self.llm.apredict(
                self.extract_prompt,
                text=text,
                max_knowledge_triplets=self.max_paths_per_chunk,
            )
            entities, entities_relationship = self.parse_fn(llm_response)
        except ValueError:
            entities = []
            entities_relationship = []

        existing_nodes = node.metadata.pop(KG_NODES_KEY, [])
        existing_relations = node.metadata.pop(KG_RELATIONS_KEY, [])
        metadata = node.metadata.copy()
        for entity, entity_type, description in entities:
            metadata[
                "entity_description"
            ] = description  # Not used in the current implementation. But will be useful in future work.
            entity_node = EntityNode(
                name=entity, label=entity_type, properties=metadata
            )
            existing_nodes.append(entity_node)

        metadata = node.metadata.copy()
        for triple in entities_relationship:
            subj, rel, obj, description = triple
            subj_node = EntityNode(name=subj, properties=metadata)
            obj_node = EntityNode(name=obj, properties=metadata)
            metadata["relationship_description"] = description
            rel_node = Relation(
                label=rel,
                source_id=subj_node.id,
                target_id=obj_node.id,
                properties=metadata,
            )

            existing_nodes.extend([subj_node, obj_node])
            existing_relations.append(rel_node)

        node.metadata[KG_NODES_KEY] = existing_nodes
        node.metadata[KG_RELATIONS_KEY] = existing_relations
        return node

    async def acall(
        self, nodes: List[BaseNode], show_progress: bool = False, **kwargs: Any
    ) -> List[BaseNode]:
        """Extract triples from nodes async."""
        jobs = []
        for node in nodes:
            jobs.append(self._aextract(node))

        return await run_jobs(
            jobs,
            workers=self.num_workers,
            show_progress=show_progress,
            desc="Extracting paths from text",
        )

## GraphRAGStore

The `GraphRAGStore` class is an extension of the `SimplePropertyGraphStore `class, designed to implement GraphRAG pipeline. Here's a breakdown of its key components and functions:


The class uses community detection algorithms to group related nodes in the graph and then it generates summaries for each community using an LLM.


**Key Methods:**

`build_communities():`

1. Converts the internal graph representation to a NetworkX graph.

2. Applies the hierarchical Leiden algorithm for community detection.

3. Collects detailed information about each community.

4. Generates summaries for each community.

`generate_community_summary(text):`

1. Uses LLM to generate a summary of the relationships in a community.
2. The summary includes entity names and a synthesis of relationship descriptions.

`_create_nx_graph():`

1. Converts the internal graph representation to a NetworkX graph for community detection.

`_collect_community_info(nx_graph, clusters):`

1. Collects detailed information about each node based on its community.
2. Creates a string representation of each relationship within a community.

`_summarize_communities(community_info):`

1. Generates and stores summaries for each community using LLM.

`get_community_summaries():`

1. Returns the community summaries by building them if not already done.

In [29]:
!pip install future

In [51]:
import re
from llama_index.core.graph_stores import SimplePropertyGraphStore
import networkx as nx
from graspologic.partition import hierarchical_leiden
from llama_index.llms.openai import OpenAI
from llama_index.core.llms import ChatMessage
from graph_rag import GraphRAGStore

## GraphRAGQueryEngine

The GraphRAGQueryEngine class is a custom query engine designed to process queries using the GraphRAG approach. It leverages the community summaries generated by the GraphRAGStore to answer user queries. Here's a breakdown of its functionality:


In [46]:
from llama_index.core.query_engine import CustomQueryEngine
from llama_index.core.llms import LLM


class GraphRAGQueryEngine(CustomQueryEngine):
    graph_store: GraphRAGStore
    llm: LLM

    def custom_query(self, query_str: str) -> str:
        """Process all community summaries to generate answers to a specific query."""
        community_summaries = self.graph_store.get_community_summaries()
        community_answers = [
            self.generate_answer_from_summary(community_summary, query_str)
            for _, community_summary in community_summaries.items()
        ]

        final_answer = self.aggregate_answers(community_answers)
        return final_answer

    def generate_answer_from_summary(self, community_summary, query):
        """Generate an answer from a community summary based on a given query using LLM."""
        prompt = (
            f"Given the community summary: {community_summary}, "
            f"how would you answer the following query? Query: {query}"
        )
        messages = [
            ChatMessage(role="system", content=prompt),
            ChatMessage(
                role="user",
                content="I need an answer based on the above information.",
            ),
        ]
        response = self.llm.chat(messages)
        cleaned_response = re.sub(r"^assistant:\s*", "", str(response)).strip()
        return cleaned_response

    def aggregate_answers(self, community_answers):
        """Aggregate individual community answers into a final, coherent response."""
        # intermediate_text = " ".join(community_answers)
        prompt = "Combine the following intermediate answers into a final, concise response."
        messages = [
            ChatMessage(role="system", content=prompt),
            ChatMessage(
                role="user",
                content=f"Intermediate answers: {community_answers}",
            ),
        ]
        final_response = self.llm.chat(messages)
        cleaned_final_response = re.sub(
            r"^assistant:\s*", "", str(final_response)
        ).strip()
        return cleaned_final_response

##  Build End to End GraphRAG Pipeline

Now that we have defined all the necessary components, let’s construct the GraphRAG pipeline:

1. Create nodes/chunks from the text.
2. Build a PropertyGraphIndex using `GraphRAGExtractor` and `GraphRAGStore`.
3. Construct communities and generate a summary for each community using the graph built above.
4. Create a `GraphRAGQueryEngine` and begin querying.

### Create nodes/ chunks from the text.

In [47]:
from llama_index.core import SimpleDirectoryReader
from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore

documents = SimpleDirectoryReader(input_files=['../data/bao-chi/gay-xuong-yte.md']).load_data()

print("length of documents:", len(documents))
print('documents:', documents[0])


from llama_index.core.node_parser import SentenceSplitter

for overlap in [5, 10, 20, 50, 100]:
    splitter = SentenceSplitter(
        chunk_size=255,
        chunk_overlap=overlap,
    )
    nodes = splitter.get_nodes_from_documents(documents)
    print(f"Chunk overlap: {overlap}, Number of nodes: {len(nodes)}")

length of documents: 1
documents: Doc ID: 63e2f37a-57fb-42e3-91bc-e89e4860a922
Text: Gãy xương không bó bột có sao không? Xương có tự lành không?  Bó
bột là phương pháp điều trị quen thuộc và phổ biến nhất với chấn
thương gãy xương. Khi bị gãy ở mức độ không quá nghiêm trọng, xương
tổn thương cần được cố định đúng cách và tránh chuyển động để thúc đẩy
quá trình hồi phục. Khi đó, bác sĩ sẽ chỉ định phương pháp bó bột. Vậy
nếu gãy...
Chunk overlap: 5, Number of nodes: 10
Chunk overlap: 10, Number of nodes: 10
Chunk overlap: 20, Number of nodes: 10
Chunk overlap: 50, Number of nodes: 10
Chunk overlap: 100, Number of nodes: 13


In [33]:
len(nodes)

13

In [34]:
print(nodes[2])

Node ID: 48f9749d-5861-463d-a423-d9b8450ac820
Text: Để trả lời cho câu hỏi gãy xương không bó bột có sao không, cùng
tìm hiểu các yếu tố ảnh hưởng đến việc điều trị gãy xương bằng cách bó
bột nhé!  Bó bột giúp lành xương gãy thế nào? Quá trình lành xương gãy
gồm 3 giai đoạn. Đầu tiên là giai đoạn các mạch máu bị tổn thương, các
tế bào miễn dịch sẽ đến vị trí tổn thương để “dọn dẹp” vụn xương gãy
...


### Build ProperGraphIndex using `GraphRAGExtractor` and `GraphRAGStore`

In [48]:
KG_TRIPLET_EXTRACT_TMPL = """
-Mục tiêu-
Cho một tài liệu văn bản, xác định tất cả các thực thể và loại thực thể của chúng từ văn bản và tất cả các mối quan hệ giữa các thực thể đã xác định.
Cho văn bản, trích xuất tối đa {max_knowledge_triplets} bộ ba thực thể-quan hệ.

-Các bước-
1. Xác định tất cả các thực thể. Đối với mỗi thực thể đã xác định, trích xuất thông tin sau:
- entity_name: Tên của thực thể, viết hoa
- entity_type: Loại của thực thể
- entity_description: Mô tả toàn diện về các thuộc tính và hoạt động của thực thể
Định dạng mỗi thực thể như sau ("entity"$$$$<entity_name>$$$$<entity_type>$$$$<entity_description>)

2. Từ các thực thể đã xác định trong bước 1, xác định tất cả các cặp (source_entity, target_entity) mà *rõ ràng có liên quan* với nhau.
Đối với mỗi cặp thực thể có liên quan, trích xuất thông tin sau:
- source_entity: tên của thực thể nguồn, như đã xác định trong bước 1
- target_entity: tên của thực thể đích, như đã xác định trong bước 1
- relation: mối quan hệ giữa thực thể nguồn và thực thể đích
- relationship_description: giải thích lý do tại sao bạn nghĩ rằng thực thể nguồn và thực thể đích có liên quan với nhau

Định dạng mỗi mối quan hệ như sau ("relationship"$$$$<source_entity>$$$$<target_entity>$$$$<relation>$$$$<relationship_description>)

3. Khi hoàn thành, xuất ra.

-Dữ liệu thực tế-
######################
text: {text}
######################
output:"""

In [36]:
import re

text = '''
("entity"$$$$Gãy Xương$$$$Medical Condition$$$$Gãy xương là tình trạng xương bị gãy do chấn thương hoặc áp lực, có thể gây đau đớn và ảnh hưởng đến khả năng vận động của bệnh nhân. Việc điều trị gãy xương có thể bao gồm bó bột, phẫu thuật, hoặc các phương pháp khác tùy thuộc vào mức độ nghiêm trọng và vị trí của gãy xương.)
'''

pattern = r'\("entity"\$\$\$\$(.+?)\$\$\$\$(.+?)\$\$\$\$(.+?)\)'

matches = re.findall(pattern, text)

print(matches)

relationship_pattern = r'\("relationship"\$\$\$\$(.+?)\$\$\$\$(.+?)\$\$\$\$(.+?)\$\$\$\$(.+?)\)'
textRelation = '''
("relationship"$$$$Gãy Xương$$$$Bó Bột$$$$Có thể điều trị bằng$$$$Gãy xương thường được điều trị bằng phương pháp bó bột để đảm bảo xương lành lại đúng cách và giảm thiểu đau đớn cho bệnh nhân.)
'''

matches = re.findall(relationship_pattern, textRelation)

print(matches)

[('Gãy Xương', 'Medical Condition', 'Gãy xương là tình trạng xương bị gãy do chấn thương hoặc áp lực, có thể gây đau đớn và ảnh hưởng đến khả năng vận động của bệnh nhân. Việc điều trị gãy xương có thể bao gồm bó bột, phẫu thuật, hoặc các phương pháp khác tùy thuộc vào mức độ nghiêm trọng và vị trí của gãy xương.')]
[('Gãy Xương', 'Bó Bột', 'Có thể điều trị bằng', 'Gãy xương thường được điều trị bằng phương pháp bó bột để đảm bảo xương lành lại đúng cách và giảm thiểu đau đớn cho bệnh nhân.')]


In [52]:
entity_pattern = r'\("entity"\$\$\$\$(.+?)\$\$\$\$(.+?)\$\$\$\$(.+?)\)'
relationship_pattern = r'\("relationship"\$\$\$\$(.+?)\$\$\$\$(.+?)\$\$\$\$(.+?)\$\$\$\$(.+?)\)'

def parse_fn(response_str: str) -> Any:
    entities = re.findall(entity_pattern, response_str)
    relationships = re.findall(relationship_pattern, response_str)
    print('parse_fn -----')
    print('entitis:', entities)
    # print('response_str:', response_str)
    return entities, relationships


kg_extractor = GraphRAGExtractor(
    llm=llm,
    extract_prompt=KG_TRIPLET_EXTRACT_TMPL,
    max_paths_per_chunk=2,
    parse_fn=parse_fn,
)

In [53]:
from llama_index.core import PropertyGraphIndex

index = PropertyGraphIndex(
    nodes=nodes,
    property_graph_store=GraphRAGStore(),
    kg_extractors=[kg_extractor],
    show_progress=True,
)

Extracting paths from text:   8%|▊         | 1/13 [00:03<00:43,  3.61s/it]

parse_fn -----
entitis: [('BÁC_SĨ', 'Người', 'Bác sĩ là người có chuyên môn trong lĩnh vực y tế, có trách nhiệm chẩn đoán và điều trị các bệnh lý, bao gồm cả việc chỉ định các phương pháp điều trị như bó bột cho bệnh nhân bị gãy xương.'), ('BÓ_BỘT', 'Phương_Pháp_Điều_Trị', 'Bó bột là một phương pháp điều trị được sử dụng để cố định xương gãy, giúp xương hồi phục và lành lại, mặc dù nó có thể gây ra sự khó chịu và bất tiện cho bệnh nhân.')]


Extracting paths from text:  15%|█▌        | 2/13 [00:04<00:19,  1.75s/it]

parse_fn -----
entitis: [('VẾT GÃY', 'THỰC THỂ', 'Một vết gãy gọn, các đầu xương gãy vẫn thẳng hàng có nhiều khả năng lành mà không cần bó bột hơn là một vết gãy xương bị di lệch, các đầu xương gãy không còn thẳng hàng.'), ('TUỔI TÁC', 'THỰC THỂ', 'Tuổi tác có thể đóng một vai trò quan trọng trong việc quyết định gãy xương không bó bột có sao không. Người trẻ tuổi có xu hướng lành vết thương nhanh hơn người lớn tuổi do xương của trẻ vẫn đang phát triển và được tưới máu tốt.')]


Extracting paths from text:  23%|██▎       | 3/13 [00:06<00:22,  2.26s/it]

...

Extracting paths from text:  85%|████████▍ | 11/13 [00:16<00:03,  1.72s/it]

parse_fn -----
entitis: [('BÁC_SĨ', 'Người', 'Bác sĩ là người có chuyên môn trong việc chẩn đoán và điều trị các chấn thương, bệnh tật và các vấn đề sức khỏe khác của bệnh nhân.'), ('BỆNH_NHÂN', 'Người', 'Bệnh nhân là người nhận sự chăm sóc y tế từ bác sĩ, có thể gặp phải các chấn thương hoặc bệnh tật cần điều trị.'), ('CHẤN_THƯƠNG', 'Sự kiện', 'Chấn thương là tình trạng tổn thương cơ thể do tác động bên ngoài, có thể dẫn đến các biến chứng như nhiễm trùng hoặc vấn đề về tưới máu.'), ('PHƯƠNG_PHÁP_ĐIỀU_TRỊ', 'Khái niệm', 'Phương pháp điều trị là các biện pháp mà bác sĩ sử dụng để chữa trị cho bệnh nhân, có thể bao gồm bó bột hoặc các phương pháp khác tùy thuộc vào tình trạng chấn thương.')]


Extracting paths from text: 100%|██████████| 13/13 [00:21<00:00,  1.67s/it]


parse_fn -----
entitis: [('BỆNH NHÂN', 'Người', 'Bệnh nhân là người nhận sự chăm sóc y tế, có thể là người bị thương hoặc mắc bệnh, và thường phải tuân thủ các hướng dẫn điều trị từ bác sĩ.'), ('BÓ BỘT', 'Thiết bị y tế', 'Bó bột là một thiết bị y tế được sử dụng để cố định và bảo vệ các bộ phận cơ thể bị thương, giúp quá trình hồi phục diễn ra thuận lợi hơn.'), ('NẸP', 'Thiết bị y tế', 'Nẹp là một thiết bị hỗ trợ y tế được sử dụng để giữ cho các bộ phận cơ thể ở vị trí cố định, thường được sử dụng trong các trường hợp chấn thương nhẹ hoặc để thay thế bó bột.'), ('VẾT THƯƠNG', 'Tình trạng y tế', 'Vết thương là tổn thương trên cơ thể do chấn thương, có thể là nhẹ hoặc nghiêm trọng, và cần được chăm sóc y tế để hồi phục.'), ('NGHỀ NGHIỆP', 'Khái niệm', 'Nghề nghiệp là công việc hoặc lĩnh vực mà một người tham gia, có thể ảnh hưởng đến sức khỏe và cách thức điều trị của họ.'), ('BỆNH NHÂN', 'Người', 'Bệnh nhân là người nhận sự chăm sóc y tế, có thể là người bị thương hoặc mắc bệnh, và thườ

Generating embeddings: 100%|██████████| 2/2 [00:02<00:00,  1.41s/it]


In [39]:
list(index.property_graph_store.graph.nodes.values())[-1]

EntityNode(label='entity', embedding=None, properties={'file_path': '../data/bao-chi/gay-xuong-yte.md', 'file_name': 'gay-xuong-yte.md', 'file_size': 5003, 'creation_date': '2024-11-03', 'last_modified_date': '2024-11-03', 'relationship_description': 'Bệnh nhân sử dụng bó bột để cố định và bảo vệ phần cơ thể bị thương trong quá trình hồi phục."', 'triplet_source_id': '2240737b-8e8c-4bb6-9428-13f506afa876'}, name='SỬ DỤNG')

In [40]:
list(index.property_graph_store.graph.relations.values())[0]

Relation(label='BÓ BỘT', source_id='GÃY XƯƠNG', target_id='ĐIỀU TRỊ', properties={'file_path': '../data/bao-chi/gay-xuong-yte.md', 'file_name': 'gay-xuong-yte.md', 'file_size': 5003, 'creation_date': '2024-11-03', 'last_modified_date': '2024-11-03', 'relationship_description': 'Gãy xương cần được điều trị để hồi phục, và bó bột là một trong những phương pháp điều trị phổ biến nhất cho tình trạng này.', 'triplet_source_id': 'd7454466-fd60-464c-8363-7ce1ed5a640a'})

In [41]:
list(index.property_graph_store.graph.relations.values())[0].properties[
    "relationship_description"
]

'Gãy xương cần được điều trị để hồi phục, và bó bột là một trong những phương pháp điều trị phổ biến nhất cho tình trạng này.'

### Build communities

This will create communities and summary for each community.

In [42]:
index.property_graph_store.build_communities()

/Users/ngocp/Documents/projects/finance-bot/.venv/lib/python3.10/site-packages/graspologic/partition/leiden.py:607: UserWarning: Leiden partitions do not contain all nodes from the input graph because input graph contained isolate nodes.
  warnings.warn(


In [43]:
index.property_graph_store.visualize_graph("y tế")

# index.property_graph_store.visualize_graph()

UnboundLocalError: local variable 'nx_graph' referenced before assignment

### Create QueryEngine

In [ ]:
query_engine = GraphRAGQueryEngine(
    graph_store=index.property_graph_store, llm=llm
)

### Querying

In [ ]:
response = query_engine.query(
    "bó bột để làm gì ?"
)
display(Markdown(f"{response.response}"))

Bó bột là phương pháp điều trị chính cho gãy xương, được sử dụng để cố định và bảo vệ xương bị thương, giúp quá trình hồi phục diễn ra hiệu quả. Việc bó bột giữ cho xương ở vị trí đúng, ngăn ngừa di chuyển và tạo điều kiện cho mạch máu và tế bào miễn dịch hoạt động, hỗ trợ quá trình lành vết thương. Quyết định bó bột phụ thuộc vào nhiều yếu tố, bao gồm vị trí gãy xương, độ tuổi của bệnh nhân, khả năng hồi phục tự nhiên và sự tuân thủ hướng dẫn điều trị. Trong một số trường hợp, như gãy xương đòn hoặc cổ tay, có thể không cần bó bột do khả năng hồi phục tốt. Tuy nhiên, ở những vị trí như xương chậu và cổ xương đùi, việc bó bột không khả thi do cấu trúc và áp lực mà các vị trí này phải chịu. Sự tuân thủ hướng dẫn điều trị và nghề nghiệp của bệnh nhân cũng có thể ảnh hưởng đến quá trình hồi phục.